In [1]:
import boto3

In [2]:
# Document
s3BucketName = "aws-textract-multi-column-pdf"
documentName = "Scanned - Multi-column Single Page Document.pdf"

In [3]:
# Amazon Textract client
textract = boto3.client('textract')

In [4]:
# Call Amazon Textract
response = textract.detect_document_text(
    Document={
        'S3Object': {
            'Bucket': s3BucketName,
            'Name': documentName
        }
    })

print(response)

{'DocumentMetadata': {'Pages': 1}, 'Blocks': [{'BlockType': 'PAGE', 'Geometry': {'BoundingBox': {'Width': 1.0, 'Height': 1.0, 'Left': 0.0, 'Top': 0.0}, 'Polygon': [{'X': 1.5849614334573464e-16, 'Y': 0.0}, {'X': 1.0, 'Y': 9.462437987838284e-17}, {'X': 1.0, 'Y': 1.0}, {'X': 0.0, 'Y': 1.0}]}, 'Id': 'e2747657-d75e-4fc0-8a3b-eec61a70f7b9', 'Relationships': [{'Type': 'CHILD', 'Ids': ['18596b46-aaf5-47a5-879e-b459760c43e1', 'c0577663-25f7-451b-94a0-8d88b1e16bd2', 'cfe31bdb-9b2d-4073-9893-301ca33e96cb', 'b0d2e83a-cd2d-4598-8e0d-c5c43a74a4a4', '9e7fff85-4b2b-415c-b6b5-2cc468295119', '342271b8-7b4d-4cad-8063-12cb28ce5f0b', '13590bb1-190a-43dc-871b-a7f40c2f5176', 'e8299215-0383-487c-9ef8-e4cc69fd838b', 'b4f8ebf4-1a1a-4662-891a-d057bf9c705b', '1cd2af71-462f-49f4-9bcd-d94bbc630adb', '2044c876-69fb-4623-ad4c-6b2c553a9369', '01146274-db84-4f68-8197-16ef1291a7b3', '53cbd341-d2e3-4e6e-ac13-6de06c3ea5e7', '3e12490a-f4d5-4e41-bf7d-c1c8b06bd704', 'a042e5d5-9c3b-4984-917e-501f46d492a4', '139be7ba-4ea9-4aba

In [5]:
# Detect columns and print lines
columns = []
lines = []
for item in response["Blocks"]:
      if item["BlockType"] == "LINE":
        column_found=False
        for index, column in enumerate(columns):
            bbox_left = item["Geometry"]["BoundingBox"]["Left"]
            bbox_right = item["Geometry"]["BoundingBox"]["Left"] + item["Geometry"]["BoundingBox"]["Width"]
            bbox_centre = item["Geometry"]["BoundingBox"]["Left"] + item["Geometry"]["BoundingBox"]["Width"]/2
            column_centre = column['left'] + column['right']/2

            if (bbox_centre > column['left'] and bbox_centre < column['right']) or (column_centre > bbox_left and column_centre < bbox_right):
                #Bbox appears inside the column
                lines.append([index, item["Text"]])
                column_found=True
                break
        if not column_found:
            columns.append({'left':item["Geometry"]["BoundingBox"]["Left"], 'right':item["Geometry"]["BoundingBox"]["Left"] + item["Geometry"]["BoundingBox"]["Width"]})
            lines.append([len(columns)-1, item["Text"]])

In [6]:
print(len(columns))

2


I added the "reverse=True" as a parameter in line.sort()

In [7]:
lines.sort(key=lambda x: x[0], reverse=True)
for line in lines:
    print (line[1])

Neque porro quisquam est
qui dolorem ipsum quia
dolor sit amet,
consectetur, adipisci
velit
Lorem ipsum dolor sit amet, consectetur
adipiscing elit, sed do eiusmod tempor
incididunt ut labore et dolore magna aliqua. Ut
enim ad minim veniam, quis nostrud
exercitation ullamco laboris nisi ut aliquip ex
ea commodo consequat. Duis aute irure dolor
in reprehenderit in voluptate velit esse cillum
dolore eu fugiat nulla pariatur. Excepteur sint
occaecat cupidatat non proident, sunt in culpa
qui officia deserunt mollit anim id est laborum.
Sed ut perspiciatis unde omnis iste natus error
sit voluptatem accusantium doloremque
laudantium, totam rem aperiam, eaque ipsa
quae ab illo inventore veritatis et quasi
architecto beatae vitae dicta sunt explicabo.
Nemo enim ipsam voluptatem quia voluptas
sit aspernatur aut odit aut fugit, sed quia
consequuntur magni dolores eos qui ratione
voluptatem sequi nesciunt. Neque porro
quisquam est, qui dolorem ipsum quia dolor
sit amet, consectetur, adipisci veli